In [16]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [17]:
seedData = pd.read_csv('data/seed_data.csv')

In [18]:
# Convert the date correctly
def getDate(r):
    time = ''
    if(r['meal'] == 'Lunch'):
        time = '12:00:00-04'
    elif(r['hospital'] == 'Hospital of the University of Pennsylvania (HUP)'):
        time = '20:30:00-04'
    elif(r['meal'] == 'Dinner'):
        time = '20:00:00-04'
    else:
        print(r)
    date = ["%02d" % int(v) for v in r['date'].split('/')]
    date = "20%s-%s-%s" % (date[2], date[0], date[1])
    return ("%s %s" % (date, time))

seedData['timeslot'] = seedData[['hospital', 'date', 'meal']].apply(getDate, axis=1)
seedData = seedData.fillna(value=False)

In [19]:
np.unique(np.concatenate([seedData['mealAllergens%i' %i].unique() for i in range(1, 5)]))

array([False, 'Dairy', 'Eggs', 'Eggs in aioli', 'Milk/dairy',
       'Pumpkin seeds (seed allergy)', 'Sesame', 'Soy', 'Soy, Sesame',
       'Soy, Sesame Oil', 'Soy,Sesame', 'Soy,Sesame Oil'], dtype=object)

In [20]:
dietLookup = {
  "V" : "Vegan",
  "VG" : "Vegetarian",
  "GF" : "Gluten Free",
  "DF" : "Dairy Free",
}
allergenLookup = {
  'Milk/dairy' : "milk/dairy",
  'Eggs' : "eggs",
  'Eggs in aioli' : 'eggs',
  'Pumpkin seeds (seed allergy)' : "pumpkin seeds",
  'Sesame' : 'sesame',
  'Sesame Oil' : 'sesame',
  'Soy' : 'soy',
  'Dairy' : 'milk/dairy'
}

In [21]:
# Build the restaurant struct
restStruct = {}

def dateCompare(t0, t1='2020-04-29 00:00:00-04'):
    v1 = datetime.strptime(t0, '%Y-%m-%d %H:%M:%S-04')
    v2 = datetime.strptime(t1, '%Y-%m-%d %H:%M:%S-04')
    return v1 > v2

def buildStruct(r):
    restName = r['restName']
    restStruct[restName] = restStruct.get(restName, {"mealIds": {}, "meals": [], "timeslots": [], "menus": []})
    # Create the timeslot and add to list
    timeslotId = str(("%s-%s-%s" % (restName, r['timeslot'], r['hospital'])).__hash__())
    timeslot = {
        "id": timeslotId,
        "hospital": r['hospital'],
        "restaurant": r['restName'],
        "date": r['timeslot']
    }
    if dateCompare(r['timeslot']):
        restStruct[restName]['timeslots'].append(timeslot) 
    # Generate the meals/menus
    for i in range(1, 5):
        mealName = r["mealName%d" % i]
        if(mealName):
            allergens, dietaryRestrictions = [], []
            if(r['mealAllergens%d' %i]):
#                 print(r['mealAllergens%d' %i])
                allergens = [v.lstrip().rstrip() for v in r['mealAllergens%d' %i].split(',')]
                allergens = [allergenLookup[a] for a in allergens]
            if(r['mealDiet%d' % i]):
#                 print(r['mealDiet%d' % i])
                dietaryRestrictions = [v.lstrip().rstrip() for v in r['mealDiet%d' % i].split('-')]
                dietaryRestrictions = [dietLookup[d] for d in dietaryRestrictions]
        
            
            # Check if meal exists else add
            if(not mealName in restStruct[r['restName']]['mealIds']):
                meal = {
                    "name": mealName,
                    "restName": restName,
                    "description": r['mealDesc%d' %i],
                    "allergens" : allergens,
                    "dietaryRestrictions": dietaryRestrictions,
                    "mealinfoId": r['meal'].lower()
                }
                restStruct[restName]['mealIds'][mealName] = meal
                restStruct[restName]['meals'].append(meal)
            # Add the menu to the list
            menu = {
                "timeslotId": timeslotId,
                "mealinfoId": r['meal'].lower(),
                "mealName": mealName,
                "mealDescription": r['mealDesc%d' %i],
                "allergens" : allergens,
                "dietaryRestrictions": dietaryRestrictions,
            }
            if dateCompare(r['timeslot']):
                restStruct[restName]['menus'].append(menu)
            
s = seedData.apply(buildStruct, axis=1)

In [22]:
dataOut = json.dumps(restStruct, indent=2)
with open("seed_data.json", "w") as outfile: 
    outfile.write(dataOut) 